In [7]:
#Source word and bigram frequency 

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path


In [2]:
mous_words = pd.read_csv('mous_words_syllable_bigram_frequencies.csv')
mous_words

,Word,IPA,Syllables,Min_Freq_Count,Max_Freq_Count,Mean_Freq_Count,Bigrams,Bigram Occurrence Counts,Zipf,FREQcount,Lg10WF
0,toen,tˈun,tun,177.0,177.0,1.770000e+02,"['to', 'oe', 'en']","[2236591.0, 837795.0, 8542439.0]",6.032333,47514.0,4.676822
1,de,dˈə,də,208410.0,208410.0,2.084100e+05,['de'],[2559661.0],7.381291,1061177.0,6.025788
2,barkeeper,bˈɑrkeːpər,bɑr - keː - pər,1201.0,22550.0,1.399933e+04,"['ba', 'ar', 'rk', 'ke', 'ee', 'ep', 'pe', 'er']","[584273.0, 1754491.0, 2174749.0, 3143026.0, 94...",3.622674,184.0,2.264818
3,die,dˈi,di,441857.0,441857.0,4.418570e+05,"['di', 'ie']","[1050254.0, 2076673.0]",6.853871,315041.0,5.498367
4,irritante,ˌɪɾritˈɑntə,ɪɾ - ri - tɑn - tə,8.0,995799.0,2.623075e+05,"['ir', 'rr', 'ri', 'it', 'ta', 'an', 'nt', 'te']","[2884214.0, 200692.0, 1787712.0, 333345.0, 892...",3.284921,84.0,1.924279
...,...,...,...,...,...,...,...,...,...,...,...
1937,blije,blˈɛɪə,blɛɪ - ə,1796.0,2281166.0,1.141481e+06,"['bl', 'li', 'ij', 'je']","[152155.0, 3369152.0, 1639122.0, 913788.0]",3.485836,134.0,2.127105
1938,jongeren,jˈɔŋərən,jɔŋ - ə - rən,6659.0,2281166.0,7.780040e+05,"['jo', 'on', 'ng', 'ge', 'er', 're', 'en']","[866375.0, 2451854.0, 2927082.0, 3692145.0, 67...",3.965097,406.0,2.608526
1939,intens,ˈɪntəns,ɪn - təns,575.0,214096.0,1.073355e+05,"['in', 'nt', 'te', 'en', 'ns']","[2264626.0, 1477178.0, 3522714.0, 8542439.0, 6...",3.778748,264.0,2.421604
1940,plezier,pleːzˈir,pleː - zir,1573.0,6109.0,3.841000e+03,"['pl', 'le', 'ez', 'zi', 'ie', 'er']","[97171.0, 4397946.0, 100963.0, 123482.0, 20766...",5.093966,5475.0,3.738384


In [3]:
mous_words.columns

Index(['Word', 'IPA', 'Syllables', 'Min_Freq_Count', 'Max_Freq_Count',
       'Mean_Freq_Count', 'Bigrams', 'Bigram Occurrence Counts', 'Zipf',
       'FREQcount', 'Lg10WF'],
      dtype='object')

In [4]:
source = Path('/media/neel/MOUS/MOUS/MOUS/SynologyDrive/source') #change this to the source directory that contains the 'transcription' CSVs 


In [7]:
for subject in source.iterdir():
    if subject.is_dir() and subject.name.startswith('sub-V'):
        print(subject.name)
        transcription = subject / 'func' / f'{subject.name}_transcription.csv'
        if transcription.exists():
            print(transcription)
            df = pd.read_csv(transcription)
            # Merge df with mous_words on 'Word'
            df = pd.merge(df, mous_words, on='Word', how='left')
            # Ensure 'Bigram Occurrence Counts' are lists of numbers
            df['Bigram Occurrence Counts'] = df['Bigram Occurrence Counts'].apply(lambda x: eval(x) if isinstance(x, str) else x)
            # Log10 of the minimum value of Bigram Occurrence Counts
            df['log10_Min_Bigram'] = df['Bigram Occurrence Counts'].apply(
                lambda x: np.log10(min(x)) if isinstance(x, list) and len(x) > 0 and all(i > 0 for i in x) else 0)
            df['log10_Min_plus1_Bigram'] = df['Bigram Occurrence Counts'].apply(
                lambda x: np.log10(1+min(x)) if isinstance(x, list) and len(x) > 0 and all(i > 0 for i in x) else 0)
            df['log10_Max_plus1_Bigram'] = df['Bigram Occurrence Counts'].apply(
                lambda x: np.log10(1+max(x)) if isinstance(x, list) and len(x) > 0 and all(i > 0 for i in x) else 0)
            df['log10_Mean_plus1_Bigram'] = df['Bigram Occurrence Counts'].apply(
                lambda x: np.log10(1+np.mean(x)) if isinstance(x, list) and len(x) > 0 and all(i > 0 for i in x) else 0)
            # Log10 of the maximum value of Bigram Occurrence Counts
            df['log10_Max_Bigram'] = df['Bigram Occurrence Counts'].apply(
                lambda x: np.log10(max(x)) if isinstance(x, list) and len(x) > 0 and all(i > 0 for i in x) else 0)
            # Log10 of the mean value of Bigram Occurrence Counts
            df['log10_Mean_Bigram'] = df['Bigram Occurrence Counts'].apply(
                lambda x: np.log10(np.mean(x)) if isinstance(x, list) and len(x) > 0 and all(i > 0 for i in x) else 0)
            df['WordLength'] = df['Word'].apply(lambda x: len(x))
            word_frequencies = df[['Onset','Word','Zipf','Lg10WF','WordLength']]
            bigrams = df[['Onset','Word','Bigrams','log10_Min_Bigram','log10_Min_plus1_Bigram','log10_Max_Bigram','log10_Mean_Bigram','WordLength','log10_Max_plus1_Bigram','log10_Mean_plus1_Bigram','log10_Mean_plus1_Bigram','log10_Max_plus1_Bigram']]
            word_frequencies.to_csv(subject / 'func' / f'{subject.name}_word_frequencies.csv', index=False)
            bigrams.to_csv(subject / 'func' / f'{subject.name}_bigrams_processed.csv', index=False)

sub-V1008
/media/neel/MOUS/MOUS/MOUS/SynologyDrive/source/sub-V1008/func/sub-V1008_transcription.csv
sub-V1081
/media/neel/MOUS/MOUS/MOUS/SynologyDrive/source/sub-V1081/func/sub-V1081_transcription.csv
sub-V1077
/media/neel/MOUS/MOUS/MOUS/SynologyDrive/source/sub-V1077/func/sub-V1077_transcription.csv
sub-V1065
/media/neel/MOUS/MOUS/MOUS/SynologyDrive/source/sub-V1065/func/sub-V1065_transcription.csv
sub-V1011
/media/neel/MOUS/MOUS/MOUS/SynologyDrive/source/sub-V1011/func/sub-V1011_transcription.csv
sub-V1086
/media/neel/MOUS/MOUS/MOUS/SynologyDrive/source/sub-V1086/func/sub-V1086_transcription.csv
sub-V1099
/media/neel/MOUS/MOUS/MOUS/SynologyDrive/source/sub-V1099/func/sub-V1099_transcription.csv
sub-V1049
/media/neel/MOUS/MOUS/MOUS/SynologyDrive/source/sub-V1049/func/sub-V1049_transcription.csv
sub-V1104
/media/neel/MOUS/MOUS/MOUS/SynologyDrive/source/sub-V1104/func/sub-V1104_transcription.csv
sub-V1100
/media/neel/MOUS/MOUS/MOUS/SynologyDrive/source/sub-V1100/func/sub-V1100_transcri

In [13]:
word_frequencies

,Onset,Word,Zipf,Lg10WF,WordLength
0,10.142,de,7.381291,6.025788,2
1,10.674,ontworpen,4.061366,2.705008,9
2,11.842,die,6.853871,5.498367,3
3,12.458,sprak,4.725275,3.369587,5
4,13.260,heeft,6.559424,5.203919,5
...,...,...,...,...,...
1375,1712.776,trompet,3.427384,2.068186,7
1376,1713.742,uitkiest,2.770476,1.397940,8
1377,1714.794,heeft,6.559424,5.203919,5
1378,1715.578,geld,5.895894,4.540380,4


In [14]:
bigrams

,Onset,Word,Bigrams,log10_Min_Bigram,log10_Min_plus1_Bigram,log10_Max_Bigram,log10_Mean_Bigram,WordLength
0,10.142,de,['de'],6.408182,6.408183,6.408182,6.408182,2
1,10.674,ontworpen,"['on', 'nt', 'tw', 'wo', 'or', 'rp', 'pe', 'en']",4.602624,4.602635,6.931582,6.331510,9
2,11.842,die,"['di', 'ie']",6.021294,6.021295,6.317368,6.194088,3
3,12.458,sprak,"['sp', 'pr', 'ra', 'ak']",5.385215,5.385216,6.346360,6.074015,5
4,13.260,heeft,"['he', 'ee', 'ef', 'ft']",5.228939,5.228942,5.975295,5.723646,5
...,...,...,...,...,...,...,...,...
1375,1712.776,trompet,"['tr', 'ro', 'om', 'mp', 'pe', 'et']",4.709914,4.709923,6.393353,6.118872,7
1376,1713.742,uitkiest,"['ui', 'it', 'tk', 'ki', 'ie', 'es', 'st']",4.368138,4.368157,6.363066,6.031377,8
1377,1714.794,heeft,"['he', 'ee', 'ef', 'ft']",5.228939,5.228942,5.975295,5.723646,5
1378,1715.578,geld,"['ge', 'el', 'ld']",5.924795,5.924795,6.669429,6.486872,4
